In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://media.githubusercontent.com/media/nhanna1219/Global-Terrorism-OLAP/main/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')

<ipython-input-54-f85cab9955c3>:1: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('https://media.githubusercontent.com/media/nhanna1219/Global-Terrorism-OLAP/main/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1')


In [ ]:
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [ ]:
columns = [
    'eventid',       # Fact_Incident: ID
    'iyear',         # Dim_Date: dYear
    'imonth',        # Dim_Date: dMonth
    'iday',          # Dim_Date: dDay

    'country',        # Dim_Country: CountryID
    'country_txt',   # Dim_Country: CountryName & Dim_Location: Country
    'region',        # Dim_Region: RegionID
    'region_txt',    # Dim_Region: region & Dim_Location: Region
    'city',          # Dim_Location: City
    'latitude',      # Dim_Location: Latitude
    'longitude',     # Dim_Location: Longitude
    'provstate',     # Dim_Location: ProvState

    'attacktype1',   # Fact_Incident: AttackTypeID
    'attacktype1_txt',# Fact_Incident: AttackType

    'targtype1',     # Fact_Incident: TargetTypeID
    'targtype1_txt', # Fact_Incident: TargetType

    'weaptype1',     # Fact_Incident: WeaponTypeID
    'weaptype1_txt',     # Fact_Incident: WeaponTypeID

    'nkill',         # Fact_Incident: Kills
    'nwound',        # Fact_Incident: Wounds
    'success',       # Fact_Incident: Success
    'suicide',       # Fact_Incident: Suicide
    'property',      # Fact_Incident: PropertyDamage

    'gname'          # Dim_GName: GName
]

df = df[columns]

In [ ]:
duplicates = df.duplicated().sum()
duplicates

0

In [ ]:
nulls = df.isnull().sum()
nulls

eventid                0
iyear                  0
imonth                 0
iday                   0
country                0
country_txt            0
region                 0
region_txt             0
city                 434
latitude            4556
longitude           4557
provstate            421
attacktype1            0
attacktype1_txt        0
targtype1              0
targtype1_txt          0
weaptype1              0
weaptype1_txt          0
nkill              10313
nwound             16311
success                0
suicide                0
property               0
gname                  0
dtype: int64

In [ ]:
# Replace null numeric values with -1
numeric_columns = ['nkill', 'nwound']
for column in numeric_columns:
    df[column] = df[column].fillna(-1)

# Replace null text values with 'N/A'
text_columns = [col for col in df.columns if col not in numeric_columns]
for column in text_columns:
    df[column] = df[column].fillna('N/A')

# Replace day/month/year with 0 value to default day/month/year(1/1/1900)
df['iyear'] = df['iyear'].replace(0, 1900)
df['imonth'] = df['imonth'].replace(0, 1)
df['iday'] = df['iday'].replace(0,1)

# Create the 'date' column by concatenating the string columns with dashes as separators
df['date'] = df['iyear'].astype(str) + '/' + df['imonth'].astype(str).str.zfill(2) + '/' + df['iday'].astype(str).str.zfill(2)
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df['date'] = df['date'].dt.strftime('%d/%m/%Y')

# Rearrange column date
columns = [col for col in df.columns if col != 'date']
dDay_index = columns.index('iday') # Insert after day column
columns.insert(dDay_index + 1, 'date')
df = df[columns]

In [ ]:
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce').fillna(0)
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce').fillna(0)

column_mapping = {
    # Dim_Date
    'eventid': 'FactID',
    'iyear': 'dYear',
    'imonth': 'dMonth',
    'iday': 'dDay',
    'date': 'dDate',

    # Dim_Location_Detail
    'country': 'CountryID',
    'country_txt': 'CountryName',
    'region': 'RegionID',
    'region_txt': 'RegionName',

    # Dim_Location
    'city': 'City',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'provstate': 'ProvState',

    # Dim_AttackType
    'attacktype1': 'AttackTypeID',
    'attacktype1_txt': 'AttackType',

    # Dim_TargetType
    'targtype1': 'TargetTypeID',
    'targtype1_txt': 'TargetType',

    # Dim_WeaponType
    'weaptype1': 'WeaponTypeID',
    'weaptype1_txt': 'WeaponType',

    # Fact_Incident
    'nkill': 'Kills',
    'nwound': 'Wounds',
    'success': 'Success',
    'suicide': 'Suicide',
    'property': 'PropertyDamage',

    # Dim_GName
    'gname': 'GName'
}

df.rename(columns=column_mapping, inplace=True)

In [ ]:
import csv
df.to_csv('Terrorism.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# Create Dimension Tables DataFrames
# dim_location_details = df[['country','country_txt','region', 'region_txt']].drop_duplicates().reset_index(drop=True)
# dim_location_details.rename(columns={'country': 'CountryID','country_txt': 'CountryName','region': 'RegionID','region_txt': 'RegionName'}, inplace=True)
# dim_location_details['DetailsID'] = (
#     dim_location_details['CountryID'].astype(str) + '_' + dim_location_details['RegionID'].astype(str)
# )

# df["DetailsID"] = dim_location_details['DetailsID']
# dim_location = df[['city', 'latitude', 'longitude', 'provstate', 'DetailsID']].drop_duplicates().reset_index(drop=True)
# dim_location.insert(0, 'LocationID', range(1, len(dim_location) + 1))
# dim_location.rename(columns={'country': 'Country', 'region': 'Region','city': 'City', 'latitude': 'Latitude', 'longitude': 'Longgitude', 'provstate': 'ProvState'}, inplace=True)

# dim_attacktype = df[['attacktype1', 'attacktype1_txt']].drop_duplicates().reset_index(drop=True)
# dim_attacktype.rename(columns={'attacktype1': 'AttackTypeID','attacktype1_txt': 'AttackType'}, inplace=True)

# dim_targettype = df[['targtype1', 'targtype1_txt']].drop_duplicates().reset_index(drop=True)
# dim_targettype.rename(columns={'targtype1': 'TargetTypeID','targtype1_txt': 'TargetType'}, inplace=True)

# dim_weapontype = df[['weaptype1', 'weaptype1_txt']].drop_duplicates().reset_index(drop=True)
# dim_weapontype.rename(columns={'weaptype1': 'WeaponID','weaptype1_txt': 'WeaponType'}, inplace=True)

# dim_gname = df[['gname']].drop_duplicates().reset_index(drop=True)
# dim_gname.insert(0, 'GNameID', range(1, len(dim_gname) + 1))
# dim_gname.rename(columns={'gname': 'GName'}, inplace=True)

# dim_date = df[['iyear', 'imonth', 'iday', 'date']].drop_duplicates().reset_index(drop=True)
# dim_date.rename(columns={'iyear': 'dYear', 'imonth': 'dMonth', 'iday': 'dDay','date': 'iDate'}, inplace=True)
# dim_date['DateID'] = pd.to_datetime({'year': dim_date['dYear'], 'month': dim_date['dMonth'], 'day': dim_date['dDay']}).dt.strftime('%Y%m%d').astype(int)
# df['dateID'] = dim_date['DateID']

# # Create Fact Table DataFrame
# df.rename(columns={'gname': 'GName'}, inplace=True)
# df = df.merge(dim_gname, how='left', on='GName')

# fact_incident = df[['eventid', 'dateID', 'attacktype1', 'targtype1', 'weaptype1', 'nkill', 'nwound', 'success', 'suicide', 'property', 'GNameID']].copy()
# fact_incident.rename(columns={
#     'dateID': "DateID",
#     'attacktype1': 'AttackTypeID',
#     'targtype1': 'TargetTypeID',
#     'weaptype1': 'WeaponTypeID',
#     'eventid': 'IncidentID',
#     'property': 'PropertyDamage',
# }, inplace=True)


# Save each DataFrame to its own CSV file

# dim_location.to_csv('dim_location.csv', index=False)
# dim_location_details.to_csv('dim_location_details.csv', index=False)
# dim_attacktype.to_csv('dim_attacktype.csv', index=False)
# dim_targettype.to_csv('dim_targettype.csv', index=False)
# dim_weapontype.to_csv('dim_weapontype.csv', index=False)
# dim_gname.to_csv('dim_gname.csv', index=False)
# dim_date.to_csv('dim_date.csv', index=False)
# fact_incident.to_csv('fact_incident.csv', index=False)
